In [11]:
using Revise
using PastaQ
using ITensors
using Zygote
using Pkg
using BenchmarkTools

In [2]:
#https://github.com/ITensor/ITensors.jl/blob/d7d6ec897586d48c624356b53054573e0286d396/src/ITensorChainRules/mps/abstractmps.jl

In [3]:
N = 1
hilbert = siteinds("Qubit", N)

1-element Vector{Index{Int64}}:
 (dim=2|id=18|"Qubit,Site,n=1")

In [4]:
os = OpSum()
os += -1, "Y", 1

H = MPO(os, hilbert)

MPO
[1] ((dim=2|id=18|"Qubit,Site,n=1")', (dim=2|id=18|"Qubit,Site,n=1"))


In [5]:
ψ = productstate(hilbert, fill(0, N))
ρs = ψ

MPS
[1] ((dim=2|id=18|"Qubit,Site,n=1"),)


In [75]:
function evolve(ρs, θ)
    gates = ITensor[]
    for i in 1:length(θ)
        if mod1(i, 2) == 1
            g = gate("Rx", siteind(ρs, 1), (θ=θ[i]))
        else
            g = gate("Rx", siteind(ρs, 1), (θ=θ[i]))
        end
        gates = vcat(gates, g)
    end
    ρ = apply(gates, ρs)
    return ρ
end

function evolve2(ρs, θ)
    ρ = ρs
    for i in 1:length(θ)
        if mod1(i, 2) == 1
            g = gate("Rx", siteind(ρs, 1), (θ=θ[i]))
        else
            g = gate("Rt", siteind(ρs, 1), (θ=θ[i]))
        end
        ρ = apply([g], ρ)
    end
    
    return ρ
end

evolve2 (generic function with 1 method)

In [86]:
function loss(θ)
    Uρ = evolve(ρs, θ)
    return real(inner(Uρ', H, Uρ))
end

function loss2(θ)
    Uρ = evolve2(ρs, θ)
    return real(inner(Uρ', H, Uρ))
end

θ = π .* 0:0.1:2
loss(θ), loss2(θ)

(0.7218500130938642, 0.7218500130938642)

In [87]:
@btime loss_grad = gradient(loss, θ)

  24.145 ms (37236 allocations: 4.23 MiB)


([-0.041860492099317914, -0.6907823519731741, -0.04165136399940923, -0.6778429499535611, -0.07913706994677437, -0.57959594328723, -0.24673205117049352, -0.3165495796314257, -0.551910602575379, -0.025237347917263557  …  -0.15943174804516128, -0.2391941188813998, -0.6445121432980633, 0.012634357962134246, -0.35763083379648253, -0.5899678859473261, 0.06492297680613793, -0.27690636458754325, -0.6291554448207154, 0.013705527055707711],)

In [88]:
@btime loss_grad = gradient(loss2, θ)

  25.337 ms (35459 allocations: 4.12 MiB)


([-0.041860492099317914, -0.6907823519731741, -0.04165136399940923, -0.6778429499535611, -0.07913706994677437, -0.57959594328723, -0.24673205117049352, -0.3165495796314257, -0.551910602575379, -0.025237347917263557  …  -0.15943174804516128, -0.2391941188813998, -0.6445121432980633, 0.012634357962134246, -0.35763083379648253, -0.5899678859473261, 0.06492297680613793, -0.27690636458754325, -0.6291554448207154, 0.013705527055707711],)

## Checking if the derivateive of the rho is correct

In [11]:
g = gate("Rx", siteind(ρs, 1), (θ=θ[1]))
g.tensor

Dim 1: (dim=2|id=826|"Qubit,Site,n=1")'
Dim 2: (dim=2|id=826|"Qubit,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2
 0.955336489125606 + 0.0im                  …                0.0 - 0.29552020666133955im
               0.0 - 0.29552020666133955im     0.955336489125606 + 0.0im

In [16]:
k = 0
loss_test = ρs -> apply([g], ρs)
loss_test_ = ρs -> apply([g], ρs)[1].tensor[k+1]
out, grad = pullback(loss_test, ρs)
ρh = productstate(hilbert, fill(k, N))
out, = grad(ρh)
for i in 1:2
    println(out[1].tensor[i])
end

s
1.0
0.0


In [13]:
for i in 1:2
    h = 1e-13
    ρh = deepcopy(ρs)
    ρh[1].tensor[i] += h
    println((loss_test_(ρh) - loss_test_(ρs))/h)
end

0.9547918011776346 + 0.0im
0.0 - 0.2955202066613396im


In [14]:
loss_test_(ρs)

0.955336489125606 + 0.0im

## Checking if the derivateive of the gates is correct

In [42]:
g = gate("Rx", siteind(ρs, 1), (θ=θ[1]))

ITensor ord=2 (dim=2|id=231|"Qubit,Site,n=1")' (dim=2|id=231|"Qubit,Site,n=1")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [83]:
k = 0
loss_test = g -> apply([g], ρs)
loss_test_ = g -> apply([g], ρs)[1].tensor[k+1]
out, grad = pullback(loss_test, g)
ρh = productstate(hilbert, fill(k, N))
out = grad(ρh)[1].tensor
for i in 1:4
    println(out[i])
end

1.0
0.0
0.0
0.0


In [84]:
for i in 1:4
    h = 1e-13
    gh = copy(g)
    gh.tensor[i] += h
    println((loss_test_(gh) - loss_test_(g))/h)
end

1.000310945187266 + 0.0im
0.0 + 0.0im
0.0 + 0.0im
0.0 + 0.0im


# Last check

In [2]:
using PastaQ
using ITensors
using Zygote
using BenchmarkTools

N = 1
hilbert = siteinds("Qubit", N)

os = OpSum()
os += -1, "Y", 1
H = MPO(os, hilbert)

ψs = productstate(hilbert, fill(0, N))

function evolve(ρs, θ)
    gates = ITensor[]
    for i in 1:length(θ)
        if mod1(i, 2) == 1
            g = gate("Rx", siteind(ρs, 1), (θ=θ[i]))
        else
            g = gate("Ry", siteind(ρs, 1), (θ=θ[i]))
        end
        gates = vcat(gates, g)
    end
    ρ = apply(gates, ρs)
    return ρ
end

function evolve2(ρs, θ)
    ρ = ρs
    for i in 1:length(θ)
        if mod1(i, 2) == 1
            g = gate("Rx", siteind(ρs, 1), (θ=θ[i]))
        else
            g = gate("Ry", siteind(ρs, 1), (θ=θ[i]))
        end
        ρ = apply([g], ρ)
    end
    
    return ρ
end

function loss(θ)
    Uψ = evolve(ψs, θ)
    return real(inner(Uψ', H, Uψ))
end

function loss2(θ)
    Uψ = evolve2(ψs, θ)
    return real(inner(Uψ', H, Uψ))
end

θ = π .* 0:0.1:2

println(loss(θ))
#0.7218500130938642
@btime loss(θ)

println(loss2(θ))
#0.7218500130938642
@btime loss2(θ)


loss_grad1 = gradient(loss, θ)
@btime gradient(loss, θ)
#24.040 ms (37236 allocations: 4.23 MiB)

loss_grad2 = gradient(loss2, θ)
@btime gradient(loss2, θ)
#25.280 ms (35459 allocations: 4.12 MiB)

loss_grad1 == loss_grad2
# true

0.7218500130938642
  768.464 μs (4144 allocations: 751.27 KiB)
0.7218500130938642
  788.731 μs (4143 allocations: 749.66 KiB)
  24.068 ms (37236 allocations: 4.23 MiB)
  25.891 ms (35459 allocations: 4.12 MiB)


true

In [5]:
using ThreadPinning


LoadError: syntax: invalid identifier name "?"

In [8]:
pinthreads(1:2:72)
threadinfo()


| 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,
  16,17,36,37,38,39,40,41,42,43,44,45,46,47,48,49,
  50,51,52,53 |
| 18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,
  34,35,54,55,56,57,58,59,60,61,62,63,64,65,66,67,
  68,69,70,71 |

# = Julia thread, # = HT, # = Julia thread on HT, | = Socket seperator

Julia threads: 36
├ Occupied CPU-threads: 36
└ Mapping (Thread => CPUID): 1 => 1, 2 => 3, 3 => 5, 4 => 7, 5 => 9, ...



In [97]:
loss_grad1 == loss_grad2

LoadError: UndefVarError: loss_grad1 not defined

In [39]:
h = 1e-11
θh = copy(θ)
θh[2] += h
(loss2(θh) - loss2(θ))/h

0.5402900349338324

In [5]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) Gold 6240 CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cascadelake)
